In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())

print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [12]:
# 데이터 전체 행의 수
total_rows = len(data)

# 0.1%에 해당하는 스레숄드 계산
threshold = total_rows * 0.001

# 'machinery' 칼럼에서 값별 빈도를 계산
machinery_counts = data['Machinery'].value_counts()

# 0.1% 이하로 등장하는 값 필터링
rare_machinery = machinery_counts[machinery_counts <= threshold].index

# 드롭: 'machinery' 값이 0.1% 이하인 행을 제외한 새로운 데이터프레임 생성
filtered_data = data[~data['Machinery'].isin(rare_machinery)]

# 필터링된 데이터 확인
print(len(filtered_data))

24050


In [3]:
data=pd.read_excel('filtered_machinery.xlsx')

In [33]:
len(data['Machinery'].unique())

76

In [4]:
import re

def preprocess_text(text):
    # 괄호 안의 내용 제거
    text = re.sub(r'\([^)]*\)', '', text)
    # 특수 문자 제거 (알파벳, 숫자, 일부 허용된 특수문자 제외)
    text = re.sub(r'[^\w\s\*\-\+/.,]', '', text)
    # 여러 공백을 언더스코어로 변환
    text = re.sub(r'\s+', '_', text)
    # 텍스트 중간의 연속된 언더스코어를 하나로 줄임
    text = re.sub(r'_+', '_', text)
    # 중간에 언더스코어가 불필요하게 남아있는 경우 처리
    text = re.sub(r'(?<!\w)_(?!\w)', '', text)
    # 언더스코어 앞뒤로 존재하는 특수문자 제거
    text = re.sub(r'_([^\w]+)_', '_', text)
    text = re.sub(r'_([^\w]+)$', '', text)
    text = re.sub(r'^([^\w]+)_', '', text)
    # 텍스트 끝부분의 불필요한 언더스코어 제거
    text = re.sub(r'_+$', '', text)
    # 영어 단어는 소문자로 변환
    text = ' '.join([word.lower() if re.match(r'[A-Za-z]', word) else word for word in text.split()])
    text = text.strip()
    return text

def clean_supplier_name(name):
    # 접미사 제거
    suffixes = r'\b(Corp\.?|Corporation|Company|Co\.?|Incorporated|Inc\.?|Limited|Ltd\.?|GmbH|S\.L\.|SDN\. BHD\.)\b'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    # 특수 문자 제거
    name = re.sub(r'[^\w\s]', '', name)
    # 불필요한 단어 제거
    name = re.sub(r'\b(사용금지|사)\b', '', name, flags=re.IGNORECASE)
    # 공백 정리
    name = re.sub(r'\s+', ' ', name).strip()
    # 오타 수정 및 문자열 정리
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name, flags=re.IGNORECASE)
    name = name.lower().strip()
    return name

In [5]:
# 각 칼럼 전처리
data['cleaned_item'] = data['청구품목'].apply(preprocess_text)
data['cleaned_supplier'] = data['발주처'].apply(clean_supplier_name)

# 전처리된 칼럼 결합
data['combined_text'] =data['cleaned_item'].fillna('') + " " + data['cleaned_supplier'].fillna('')


In [6]:
print(data[['combined_text']])

                                     combined_text
0      ge_power_pack_fork_e7 matsuiusa corporation
1      ge_power_pack_fork_e7 matsuiusa corporation
2                  h-ex_30_4_1/4,_100md_120fms kti
3                  nylon_54_4_1/4,_100md_50fms kti
4                  nylon_48_4_1/4,_100md_50fms kti
...                                            ...
24045             ring-o haein corporation_cheonan
24046     ring-retaining haein corporation_cheonan
24047     sleeve-bearing haein corporation_cheonan
24048       bearing-ball haein corporation_cheonan
24049    bearing-ball_de haein corporation_cheonan

[24050 rows x 1 columns]


In [7]:
from gensim.models import FastText, Word2Vec
import torch

# 문장을 토큰화하여 리스트로 만들어야 합니다.
sentences = [text.split() for text in data['combined_text']]

# FastText 모델 학습
ft_model = FastText(vector_size=100, window=5, min_count=1, min_n=3, max_n=6, sg=1)
ft_model.build_vocab(sentences)
ft_model.train(sentences, total_examples=len(sentences), epochs=10)

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)
w2v_model.train(sentences, total_examples=len(sentences), epochs=10)

(353290, 712710)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
# FastText 임베딩을 가져오는 함수
def get_embedding(word, model):
    if word in model.wv:
        return torch.tensor(model.wv[word])
    else:
        # 서브워드 임베딩의 평균을 계산
        subwords = [word[i:j] for i in range(len(word)) for j in range(i+1, len(word)+1)]
        subword_vectors = [model.wv[subword] for subword in subwords if subword in model.wv]
        
        if subword_vectors:
            return torch.tensor(subword_vectors).mean(dim=0)
        else:
            return torch.zeros(model.vector_size)  # 단어가 없는 경우 0 벡터로 처리
# FastText 임베딩과 Word2Vec 임베딩을 결합한 함수
def get_combined_embedding(word, ft_model, w2v_model):
    ft_vector = get_embedding(word, ft_model)  # FastText에서 얻은 임베딩
    if word in w2v_model.wv:
        w2v_vector = torch.tensor(w2v_model.wv[word])  # Word2Vec에서 얻은 임베딩
    else:
        w2v_vector = torch.zeros(w2v_model.vector_size)  # 단어가 없는 경우 0 벡터로 처리

    combined_vector = torch.cat((ft_vector, w2v_vector))  # 두 임베딩을 결합 (concatenate)
    return combined_vector

# 결합된 임베딩을 생성
combined_embeddings = []
for text in data['combined_text']:
    words = text.split()
    word_vectors = [get_combined_embedding(word, ft_model, w2v_model) for word in words]
    if word_vectors:
        embedding = torch.stack(word_vectors).mean(dim=0)  # 단어 벡터의 평균 계산
    else:
        embedding = torch.zeros(model.vector_size + w2v_model.vector_size)  # 단어가 없는 경우 0 벡터로 처리
    combined_embeddings.append(embedding)

# 결합된 임베딩 리스트를 텐서로 변환
combined_embeddings_tensor = torch.stack(combined_embeddings)

print(combined_embeddings_tensor.shape)  # 결합된 임베딩 텐서

torch.Size([24050, 200])


In [10]:
import torch
import torch.nn.functional as F

# 모든 단어에 대해 결합된 임베딩을 계산하고 저장
combined_word_vectors = {}
for word in ft_model.wv.index_to_key:  # FastText 모델의 모든 단어에 대해 반복
    combined_word_vectors[word] = get_combined_embedding(word, ft_model, w2v_model)

# 특정 단어와 가장 유사한 단어 5개를 찾는 함수 정의
def find_similar_words(target_word, combined_word_vectors, topn=5):
    if target_word not in combined_word_vectors:
        print(f"Word '{target_word}' not in vocabulary.")
        return []

    target_vector = combined_word_vectors[target_word]
    similarities = {}

    for word, vector in combined_word_vectors.items():
        similarity = F.cosine_similarity(target_vector.unsqueeze(0), vector.unsqueeze(0)).item()
        similarities[word] = similarity

    # 상위 topn개의 유사 단어를 찾음
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return sorted_similarities[:topn]

In [13]:
# 특정 단어의 유사 단어 찾기
word = "valve".lower()   # 여기에는 확인하고 싶은 단어를 넣으세요
similar_words = find_similar_words(word, combined_word_vectors, topn=5)

# 결과 출력
print(f"Words most similar to '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")

Words most similar to 'valve':
valve: 1.0000
check_valve: 0.9829
cover_gasket: 0.9822
connector: 0.9821
valve_sun_cbcg-ljn: 0.9804


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 1. 데이터 준비 및 인코딩
machinery = data['Machinery'].values
machinery_encoder = LabelEncoder()
machinery_labels = machinery_encoder.fit_transform(machinery)

# 결합된 임베딩 텐서를 넘파이 배열로 변환
X = combined_embeddings_tensor.numpy()

# 2. Train-Test Split (각 레이블에 대해 동일한 분할 사용)
X_train_val, X_test, y_train_val_machinery, y_test_machinery = train_test_split(
    X, machinery_labels, test_size=0.2, random_state=42, stratify=machinery_labels
)

X_train, X_val, y_train_machinery, y_val_machinery = train_test_split(
    X_train_val, y_train_val_machinery, test_size=0.2, random_state=42, stratify=y_train_val_machinery
)


# 4. 데이터 정규화 (StandardScaler)
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)  # 검증 데이터 정규화
X_test_normalized = scaler.transform(X_test)  # 테스트 데이터 정규화


# 5. Train 데이터를 torch Tensor로 변환
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_normalized, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)

y_train_machinery_tensor = torch.tensor(y_train_machinery, dtype=torch.long).to(device)
y_val_machinery_tensor = torch.tensor(y_val_machinery, dtype=torch.long).to(device)
y_test_machinery_tensor = torch.tensor(y_test_machinery, dtype=torch.long).to(device)


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

# Transformer 기반 모델 정의
class MachineryTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, output_dim, dropout=0.3):
        super(MachineryTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)  # 입력 차원을 Transformer 입력 차원(hidden_dim)으로 변환
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)  # 최종 분류를 위한 출력 레이어

    def forward(self, x):
        x = self.embedding(x)  # 입력을 hidden_dim 크기로 변환
        x = x.unsqueeze(1)  # 시퀀스 차원 추가 -> (batch_size, sequence_length=1, hidden_dim)
        x = self.transformer_encoder(x)  # Transformer 인코더 레이어
        x = x.mean(dim=1)  # 각 시퀀스의 평균을 사용해 문장 임베딩을 생성
        x = self.fc(x)  # 분류를 위한 출력 레이어
        return x

In [28]:
# 모델 설정
input_dim = X_train_tensor.shape[1]  # 임베딩 차원 (200)
hidden_dim = 256  # Transformer 은닉 상태 크기
num_heads = 4  # Multi-head attention의 head 수
num_layers = 2  # Transformer 레이어 수
output_dim = len(machinery_encoder.classes_)  # 분류할 클래스의 수

model = MachineryTransformer(input_dim, hidden_dim, num_heads, num_layers, output_dim).to(device)


In [29]:
print(model)

MachineryTransformer(
  (embedding): Linear(in_features=200, out_features=128, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=128, out_features=76, bias=True)
)


In [30]:
# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 분류 문제이므로 CrossEntropyLoss 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저 사용


In [31]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 1. TensorDataset과 DataLoader로 데이터셋을 배치 단위로 묶음
train_dataset = TensorDataset(X_train_tensor, y_train_machinery_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_machinery_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_machinery_tensor)

# DataLoader로 배치 단위로 데이터를 나누기
batch_size = 64  # 원하는 배치 크기
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [32]:
# 학습 루프 정의
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # 검증 데이터에서 성능 확인
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                val_outputs = model(X_batch)
                val_loss += criterion(val_outputs, y_batch).item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}")


In [33]:
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)


Epoch [1/10], Loss: 457.3410, Val Loss: 101.7729
Epoch [2/10], Loss: 399.5066, Val Loss: 94.1813
Epoch [3/10], Loss: 382.7635, Val Loss: 91.9652
Epoch [4/10], Loss: 368.9719, Val Loss: 90.3656
Epoch [5/10], Loss: 363.8580, Val Loss: 88.8540
Epoch [6/10], Loss: 358.8641, Val Loss: 89.5257
Epoch [7/10], Loss: 353.6300, Val Loss: 88.4274
Epoch [8/10], Loss: 349.4143, Val Loss: 85.3485
Epoch [9/10], Loss: 346.3661, Val Loss: 85.0262
Epoch [10/10], Loss: 343.2732, Val Loss: 86.0768
